### Preparando entorno

In [1]:
import enum
print(enum.__file__)  
# standard library location should be something like 
# /usr/local/lib/python3.6/enum.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/enum.py


In [2]:
import sys
!{sys.executable} -m pip uninstall -y enum34

In [3]:
!{sys.executable} -m pip install awswrangler

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [5]:
!{sys.executable} -m pip install geopandas

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### Configurar PyAthena

In [6]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
from datetime import datetime
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [7]:
import geopandas as gpd
import glob
import os

In [8]:
import awswrangler as wr
import boto3
s3 = boto3.resource('s3')

### Seleccionar paises

In [9]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [10]:
pais = 'BR'

In [11]:
cat_adm = 'mun'
niv_adm = '4'

### Generando tabla de pois

In [12]:
# Creando tabla de adm

table_poi = f'{pais}_adm{niv_adm}'

query = f'''
        CREATE external TABLE IF NOT EXISTS {table_poi}
         (
         idn double,
         boundary binary
         )
        ROW FORMAT SERDE 'com.esri.hadoop.hive.serde.JsonSerde'
        STORED AS INPUTFORMAT 'com.esri.json.hadoop.EnclosedJsonInputFormat'
        OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
        LOCATION 's3://iadbprod-csd-hub-analyticaldata/Shapefiles_adms/json/{cat_adm}/{pais}/'
            '''
cursor.execute(query)

print(f'Generacion de tabla {table_poi}: terminado')

Generacion de tabla BR_adm4: terminado


#### Borrar tablas

pais = 'PE'
try:
    
    tabla_datos_pais = f"{pais}_adm{niv_adm}"
    query = f"DROP TABLE {tabla_datos_pais}"
    cursor.execute(query)
        
except:

    print(f" Eliminado ")